In [377]:
import numpy as np
import statsmodels.api as sm
import copy

In [378]:
PRECISION = 4

In [379]:
def formatTabular(arr) -> None:
    arr = np.array(arr)
    # 1d array
    if len(arr.shape) == 1:
        for ind, el in enumerate(arr):
            print(round(el, PRECISION), end=' & ') if ind != len(arr) - 1 else print(round(el, PRECISION), end=' \\\\ \n \\hline \n')
    # 2d array
    elif len(arr.shape) == 2:
        for row in arr:
            for ind, el in enumerate(row):
                print(round(el, PRECISION), end=' & ') if ind != len(row)- 1 else print(round(el, PRECISION), end=' \\\\ \n \\hline \n')
    
    else:
        raise Exception("Invalid array dimension")

In [380]:
data = """
1,158574 1,194067 1,745872 1,566271 1,825556 1,942503
1,238868 1,913419 1,182653 1,044649 1,304209 1,924039
1,564043 1,561357 1,070589 1,778954 1,226447 1,824122
1,737266 1,798975 1,952239 1,752281 1,247871 1,54796
1,364544 1,03122 1,380596 1,688101 1,987396 1,058504
1,535295 1,742973 1,580401 1,063356 1,999237 1,425459
1,780725 1,306711 1,972594 1,68627 1,582629 1,767235
1,135044 1,139164 1,686178 1,220069 1,034577 1,019745
1,246498 1,114597 1,079653 1,333415 1,054445 1,156743
1,416456 1,349223 1,68038 1,003235 1,471908 1,095523
1,611866 1,972991 1,443953 1,014008 1,91699 1,182531
1,520585 1,427992 1,464156 1,011505 1,108341 1,981536
1,229896 1,304392 1,852107 1,705496 1,725639 1,21482
1,726829 1,866756 1,074984 1,09888 1,983154 1,256935
1,77279 1,363353 1,227454 1,076754 1,656758 1,675253
1,418256 1,072481 1,123447 1,438917 1,059481 1,080325
1,119724 1,947356 1,372631 1,635578 1,94058 1,112827
1,728446 1,802332 1,365001 1,184759 1,119633 1,880032
1,161107 1,359294 1,956206 1,143406 1,49144 1,688437
1,963561 1,271859 1,250008 1,19367 1,466262 1,624409
"""

data = data.strip().split('\n')
data = [list(map(float, line.replace(',', '.').split())) for line in data]
# data = np.array(data)
dataCopy = copy.deepcopy(data)
# data

In [381]:
N = 9
# N = 8

n = 52
alpha = (n - 53)/40

print(f'N = {N}, alpha = {alpha}')

N = 9, alpha = -0.025


In [382]:
# y + alpha
additionalData = [14.77, 13.41, 13.84, 13.60, 13.23, 14.88, 15.39, 9.56, 10.37, 11.95, 14.13, 13.83, 12.51, 14.9, 15.31, 10.67, 12.52, 14.18, 13.02, 15.16]
# additionalData = [14.65, 15.06, 13.37, 15.90, 10.41, 14.25, 15.14, 11.54, 10.05, 12.38, 13.80, 14.62, 13.11, 12.60, 12.80, 9.82, 13.30, 15.14, 15.01, 12.44]

# y 
y = [round(el - alpha, 3) for el in additionalData]

# y

In [383]:
if len(additionalData) == len(y):
    for i in range(len(y)):
        dataCopy[i].extend([additionalData[i], y[i]])
else:
    raise Exception("Ivalid data")

dataCopy

[[1.158574, 1.194067, 1.745872, 1.566271, 1.825556, 1.942503, 14.77, 14.795],
 [1.238868, 1.913419, 1.182653, 1.044649, 1.304209, 1.924039, 13.41, 13.435],
 [1.564043, 1.561357, 1.070589, 1.778954, 1.226447, 1.824122, 13.84, 13.865],
 [1.737266, 1.798975, 1.952239, 1.752281, 1.247871, 1.54796, 13.6, 13.625],
 [1.364544, 1.03122, 1.380596, 1.688101, 1.987396, 1.058504, 13.23, 13.255],
 [1.535295, 1.742973, 1.580401, 1.063356, 1.999237, 1.425459, 14.88, 14.905],
 [1.780725, 1.306711, 1.972594, 1.68627, 1.582629, 1.767235, 15.39, 15.415],
 [1.135044, 1.139164, 1.686178, 1.220069, 1.034577, 1.019745, 9.56, 9.585],
 [1.246498, 1.114597, 1.079653, 1.333415, 1.054445, 1.156743, 10.37, 10.395],
 [1.416456, 1.349223, 1.68038, 1.003235, 1.471908, 1.095523, 11.95, 11.975],
 [1.611866, 1.972991, 1.443953, 1.014008, 1.91699, 1.182531, 14.13, 14.155],
 [1.520585, 1.427992, 1.464156, 1.011505, 1.108341, 1.981536, 13.83, 13.855],
 [1.229896, 1.304392, 1.852107, 1.705496, 1.725639, 1.21482, 12.51, 12.5

In [384]:
X = np.array(data) # Независимые переменные
Y = np.array(y)    # Зависимая   переменные

# Добавляем константу (свободный член β0) для включения интерсепта
X = sm.add_constant(X)

# Создание и обучение модели
model = sm.OLS(Y, X)
results = model.fit()

# results.summary()
print("Коэффициенты:\n", [round(el, 9) for el in results.params], end='\n\n')
print("t-статистика:\n", [round(el, 9) for el in results.tvalues], end='\n\n')

intervals = results.conf_int()
lowerIntervals = []
upperIntervals = []
# print("Доверительные интервалы (95%):\n", , end='\n\n')
for el in intervals:
    lowerIntervals.append(el[0])
    upperIntervals.append(el[1])

print("Нижние 95%:\n", [round(el, 9) for el in lowerIntervals], end='\n\n')
print("Верхние 95%:\n", [round(el, 9) for el in upperIntervals], end='\n\n')


Коэффициенты:
 [0.017020047, 2.998635451, 0.006189067, -0.001028465, 0.000461738, 2.999563826, 3.00008523]

t-статистика:
 [1.792948188, 820.621892592, 2.004150255, -0.343552884, 0.145094937, 1091.222032835, 1062.342042503]

Нижние 95%:
 [-0.003487838, 2.990741245, -0.000482422, -0.007495775, -0.00641324, 2.99362538, 2.993984285]

Верхние 95%:
 [0.037527932, 3.006529657, 0.012860557, 0.005438846, 0.007336716, 3.005502273, 3.006186175]



In [385]:
print('Коэффициенты & ',end='')
formatTabular(results.params)
print('t-статистика & ',end='')
formatTabular(results.tvalues)
print('Нижние 95\% & ',end='')
formatTabular(lowerIntervals)
print('Верхние 95\% & ',end='')
formatTabular(upperIntervals)

Коэффициенты & 0.017 & 2.9986 & 0.0062 & -0.001 & 0.0005 & 2.9996 & 3.0001 \\ 
 \hline 
t-статистика & 1.7929 & 820.6219 & 2.0042 & -0.3436 & 0.1451 & 1091.222 & 1062.342 \\ 
 \hline 
Нижние 95\% & -0.0035 & 2.9907 & -0.0005 & -0.0075 & -0.0064 & 2.9936 & 2.994 \\ 
 \hline 
Верхние 95\% & 0.0375 & 3.0065 & 0.0129 & 0.0054 & 0.0073 & 3.0055 & 3.0062 \\ 
 \hline 
